In [185]:
import pandas as pd
import glob
import os
import pickle

### System-level data

In [186]:
wmt19_no_metadata = pickle.load(open('../data/pickles/wmt19-sys_level-all.pkl', 'rb'))
wmt19_no_metadata.count()

system       129126
sid          129126
raw_score    129126
score        129126
N            129126
lp           129126
reference    129126
source       129126
output       123700
dtype: int64

In [187]:
df = pd.read_csv('./data/newstest2019-humaneval/mturk-sntlevel-humaneval-newstest2019/analysis/ad-good-raw-redup.csv', sep='\t')
df.count()

HITId         131720
WorkerId      131720
Input.src     131720
Input.trg     131720
Input.item    131720
hit           131720
sys_id        131720
rid           131720
type          131720
sid           131720
score         131720
time          131720
dtype: int64

In [188]:
# turkle
turkle_df = pd.read_csv('./data/newstest2019-humaneval/turkle-sntlevel-humaneval-newstest2019/analysis/ad-good-raw-redup.csv', sep='\t')
turkle_df['sys_id'] = turkle_df.sys_id.apply(lambda x: x[:-6])
turkle_df.drop('hit', axis=1, inplace=True)
df = pd.concat([df, turkle_df])

In [178]:
df['lp'] = df.apply(lambda x: '%s-%s' % (x['Input.src'], x['Input.trg']), axis=1)

# fix unsupervised
df['sys_id'] = df.apply(lambda x: 'Unsupervised.%s' % x['sys_id'].split('.')[-1] if 'Unsupervised.' in x['sys_id'] else x['sys_id'], axis=1)

df.head(1)

,HITId,WorkerId,Input.src,Input.trg,Input.item,hit,sys_id,rid,type,sid,score,time,lp
0,3MA5N0ATTCBZLJ39L2ULJO4IBBWWK2,M0765,fi,en,ad,1.0,parfda.6526,0,SYSTEM,367,71,626,fi-en


In [180]:
wmt19_metadata = df.merge(wmt19_no_metadata[['system', 'sid', 'lp', 'reference', 'source', 'output']],
         left_on=['lp', 'sys_id', 'sid'],
         right_on=['lp', 'system', 'sid'],
         how='left'
        )
wmt19_metadata.count()

HITId         179519
WorkerId      179519
Input.src     179519
Input.trg     179519
Input.item    179519
hit           131720
sys_id        179519
rid           179519
type          179519
sid           179519
score         179519
time          179519
lp            179519
system        161714
reference     161714
source        161714
output        161714
dtype: int64

In [183]:
wmt19_metadata[wmt19_metadata.system.isna()].lp.value_counts()

zh-en    6599
de-fr    4269
de-cs    3859
fr-de    3078
Name: lp, dtype: int64

In [184]:
wmt19_metadata = wmt19_metadata[['lp', 'HITId', 'WorkerId', 'score', 'time', 'system', 'type', 'sid', 'reference', 'source', 'output']]

In [53]:
pickle.dump(wmt19_metadata, open('pickles/wmt19_sys_metadata.pkl', 'wb'))